# Tile Generation Benchmarks using CMR

## Explanation

In this notebook we explore tile generation using titiler-cmr with the GPM IMERG dataset.

In [1]:
import cf_xarray  # noqa
import earthaccess
import fsspec
import hvplot.pandas  # noqa
import morecantile
import xarray as xr
from cmr_tile_test import CMRTileTest

import helpers.dataframe as dataframe_helpers
import helpers.eodc_hub_role as eodc_hub_role

## Set parameters for benchmarks

In [2]:
concept_id = "C2723754850-GES_DISC"  # CMR concept-id for GPM-IMERG
variable = "precipitation"
cmr_query = {"temporal": ("2024-02-11", "2024-02-11")}
iterations = 10

## Define utility functions for running tests

In [3]:
def load_earthaccess_data(concept_id, cmr_query):
    """Load dataset associated with concept idea and time range using earthaccess"""
    results = earthaccess.search_data(
        concept_id=concept_id, cloud_hosted=True, temporal=cmr_query["temporal"]
    )
    with fsspec.open(results[0].data_links("direct")[0]) as f:
        ds = xr.open_dataset(f)
    return ds


def get_resolution(ds):
    """Return a rough estimate of the dataset resolution"""
    return max(ds.lat.diff(dim="lat").max().values, ds.lon.diff(dim="lon").max().values)


def get_webmercator_extent(ds):
    """Return the dataset area extent, cropped to the bounds of the WebMercatorQuad"""
    return {
        "lon_extent": [int(ds.lon.min().values), int(ds.lon.max().values)],
        "lat_extent": [
            max(-85, int(ds.lat.min().values)),
            min(85, int(ds.lat.max().values)),
        ],
    }


def get_max_zoom_level(resolution):
    """Return the WebMercatorQuad zoom level associated with the full dataset resolution"""
    tms = morecantile.tms.get("WebMercatorQuad")
    return tms.zoom_for_res(resolution, max_z=30, zoom_level_strategy="auto")


def generate_arguments(cmr_test, iterations, max_zoom_level):
    """Return titiler-cmr queries for generating random tile indices over all requested zoom levels"""
    arguments = []
    zoom_levels = range(max_zoom_level)
    for zoom in zoom_levels:
        arguments.extend(cmr_test.generate_arguments(iterations, zoom))
    return arguments

In [4]:
def run_cmr_tile_tests(concept_id, cmr_query, variable):
    """Generate tiles using titiler-cmr. Returns the URI to the results"""
    ds = load_earthaccess_data(concept_id, cmr_query)
    resolution = get_resolution(ds)
    extent = get_webmercator_extent(ds)
    max_zoom_level = get_max_zoom_level(resolution)
    cmr_test = CMRTileTest(
        dataset_id=concept_id,
        lat_extent=extent["lat_extent"],
        lon_extent=extent["lon_extent"],
        cmr_query=cmr_query,
        variable=variable,
    )
    arguments = generate_arguments(cmr_test, iterations, max_zoom_level)
    cmr_test.run_batch(arguments=arguments)
    credentials = eodc_hub_role.fetch_and_set_credentials()
    return cmr_test.store_results(credentials)

## Run the tests

In [5]:
cmr_test_results = []
cmr_test_results.append(run_cmr_tile_tests(concept_id, cmr_query, variable))

Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules found: 1
Granules f

## Load + plot results 

In [6]:
credentials = eodc_hub_role.fetch_and_set_credentials()
df = dataframe_helpers.load_all_into_dataframe(
    credentials, cmr_test_results, use_boto3=False
)
df = dataframe_helpers.expand_timings(df)

In [7]:
df.hvplot.box(
    y="time",
    by=["zoom"],
    ylabel="Time to tile (ms)",
    xlabel="Zoom level",
    legend=False,
)

:BoxWhisker   [zoom]   (time)